# Filters detected blob-crops
- edge detector filter only
- size filter not applied
- can merge input npy files of the same name pattern, but don't use it, loc_czi map will be disconnected (one crops file a time)

- instruction:
    - put under res/filter_merge/ folder
    - don't run two instances under the same folder

In [ ]:
#%matplotlib inline
import sys
from pathlib import Path
home = str(Path.home())
sys.path.append(home+'/ccount/ccount/workflow/scripts')
sys.path.append(home+'/ccount/ccount/workflow/scripts/ccount')
import glob
import numpy as np
import matplotlib.pyplot as plt
from ccount_utils.blob import load_blobs
from ccount_utils.blob import show_rand_crops
import ccount_utils.blob # for remove_edge_crops
from ccount_utils.blob import flat2image

In [ ]:
%pwd

In [ ]:
%ls "../blob_crops"

# Load Pre-build crops (cropped images of blobs)

In [ ]:
name = "E2f4_CFUe_NoEpo_1-Stitching-04.0"
fname_pattern = '../blob_crops/' + name + '.crops.npy.gz'
[x for x in glob.iglob(fname_pattern)]

In [ ]:
i = 0
for fname in glob.iglob(fname_pattern):
    i+=1
    print(i, fname)
    if (i== 1):
        crops = load_blobs(fname)
    else:
        crop_ = load_blobs(fname)
        crops = np.vstack((crops, crop_))
    print('merged crops:', crops.shape)

# Remove very small blobs

In [ ]:
area_before = crops[:, 6]

idx_small = crops[:, 6] <= 2e4
show_rand_crops(crops[idx_small,], num_shown=10)

idx = crops[:, 6] > 2e4
crops = crops[idx, ]
print(round(1-sum(idx)/len(idx), 2), 'removed')

plt.hist(area_before, 40, alpha=0.5, label = 'before filtering')
plt.hist(crops[:, 6], 40, alpha=0.5, label = 'after filtering')
plt.title("hist of blob area size in pixels")
plt.legend()
plt.show()

## Remove blobs with edges in the crop

In [ ]:
from importlib import reload  
reload(ccount_utils.blob)
crops_clean, crops_bad = ccount_utils.blob.remove_edge_crops(crops)
print(len(crops_clean), len(crops_bad), round(len(crops_bad)/len(crops), 2))

In [ ]:
area_bad = crops_bad[:,6]
area_clean = crops_clean[:,6]

plt.hist(area_bad, 40, alpha = 0.5, label = 'bad')
plt.hist(area_clean, 40 , alpha = 0.5, label = 'clean')
plt.legend(loc='upper right')
plt.title("Histogram of blob size in pixcels")
plt.show()

In [ ]:
show_rand_crops(crops_bad, num_shown=4)

In [ ]:
show_rand_crops(crops_clean, num_shown=10)

In [ ]:
np.save(name+'.flt', crops_clean)

In [ ]:
!gzip -f *npy

In [ ]:
%ls -lrth

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:
!jupyter nbconvert --to html filter_merge.ipynb

In [ ]:
nb_name = name + '.filter_merge.ipynb'
html_name = name + ".filter_merge.html"
!echo $html_name $nb_name
!cp filter_merge.ipynb $nb_name
!mv filter_merge.html $html_name